# Module Imports

In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy.interpolate import griddata


In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

In [3]:
## IMPORT REGRESSSION MODULES
from sys import platform,path
import getpass
username = getpass.getuser()
if platform == 'darwin':
    module_path = '/Users/'+username+'/catkin_ws/src/crazyflie_simulation/src/Projects/IROS_DataAnalysis'
else:
    module_path = '/home/'+username+'/catkin_ws/src/crazyflie_simulation/src/Projects/IROS_DataAnalysis'
path.append(module_path)

from DataVisualizer.LinReg import plot_3d_data,linreg,reg_3d,reg_2d,color_plot

# DataFrame

In [4]:
## FULL DATAFRAME
df_raw = pd.read_csv("NS_2-Policy_Summary.csv")

## DROP EXTRANEOUS DATAPOINTS
df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vz_d']>=0)].index)
df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vx_d']>=0.0)].index)


## GROUP AND AVERAGE RAW DATA BY VEL IC
df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

## CLEAN RAW DATA
df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
# df['My_d'] = df['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max moment
# df = df.drop(df[(df['vz_d']<= 2.0) & (df['vx_d']<= 0.5)].index) # Drop corner with no successful landings
df = df.dropna()

## MAXIMUM DATAFRAME
idx = df.groupby(['vz_d','vx_d'])['landing_rate'].transform(max) == df['landing_rate']
df_max = df[idx].reset_index()

## MY OUTLIER REMOVED DATAFRAME
df_max = df_max.query(f"My_d<={7.7}")
df_max = df_max.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_max = df_max.query(f"landing_rate>={0.6}") # consider only top x landing rates

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

# Landing Rate Data

In [5]:
## LANDING RATE DATA (MAX)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

x,y,z = 'vx_d', 'vz_d', 'landing_rate'
# DEFINE VARIABLES
X = df_max[x]
Y = df_max[y]
Z = df_max[z]


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)
ax.set_ylim(1.5,4)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Max Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.57
Equation:  landing_rate  =  0.724  +  0.107 * vx_d  +  -0.01 * vz_d


In [7]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.5651
Equation:  landing_rate  =  0.692  +  0.109 * vx_d


vz_d results:
r2 =  0.0458
Equation:  landing_rate  =  0.957  +  -0.031 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RREV vs IC

In [8]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
x,y,z = 'vx_d','vz_d','RREV_threshold'

X = df_max[x]
Y = df_max[y]
Z = df_max[z]
C = df_max[z]


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=2)
#ax.plot_trisurf(X,Y,ypred,alpha=0.75,label='Linear_Regression',zorder=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")



# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Max Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.2814
Equation:  RREV_threshold  =  4.841  +  0.099 * vx_d  +  -0.155 * vz_d


In [10]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.1178
Equation:  RREV_threshold  =  4.355  +  0.129 * vx_d


vz_d results:
r2 =  0.2145
Equation:  RREV_threshold  =  5.057  +  -0.175 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [11]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


# DEFINE VARIABLES
x,y,z = 'RREV_threshold','OF_y','impact_tdelta'
X = df_max[x]
Y = df_max[y]
Z = df_max[z]
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
# # fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Max Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.7197
Equation:  impact_tdelta  =  0.7  +  -0.103 * RREV_threshold  +  0.001 * OF_y


In [13]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

RREV_threshold results:
r2 =  0.7192
Equation:  impact_tdelta  =  0.707  +  -0.105 * RREV_threshold


OF_y results:
r2 =  0.2462
Equation:  impact_tdelta  =  0.257  +  0.01 * OF_y


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [14]:
## Define Dataframe

#df_temp = df.query(f"landing_rate>={0.1}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


x,y,z = 'vx_d','vz_d','My_d'
X = df_max[x]
Y = df_max[y]
Z = df_max[z]
pcm = ax.scatter(X,Y,Z)



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d [N*mm]')

ax.set_title('My_d vs IC (Max Data)')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.09
Equation:  My_d  =  7.156  +  -0.361 * vx_d  +  -0.556 * vz_d


In [16]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.016
Equation:  My_d  =  5.415  +  -0.254 * vx_d


vz_d results:
r2 =  0.0588
Equation:  My_d  =  6.371  +  -0.486 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"impact_eul<={-60}")  
#df_temp = df_max

# DEFINE VARIABLES
x,y,z = 'vx_d','vz_d','impact_eul'
X = df_temp[x]
Y = df_temp[y]
Z = -df_temp[z]
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle [deg]')

ax.set_title('Impact Angle vs IC (Max Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.0816
Equation:  impact_eul  =  113.739  +  0.02 * vx_d  +  7.853 * vz_d


In [19]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

vx_d results:
r2 =  0.003
Equation:  impact_eul  =  138.328  +  -1.497 * vx_d


vz_d results:
r2 =  0.0816
Equation:  impact_eul  =  113.783  +  7.849 * vz_d


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# MAX POLICY RELATION

In [20]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


# DEFINE VARIABLES
x,y,z = 'RREV_threshold','OF_y','My_d'
X = df_max[x]
Y = df_max[y]
Z = df_max[z]
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Max Policy Relation')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
## 3D REGRESSION ANALYSIS AND PLOTTING

# Regression results
xval, yval, zval, mv_reg, x_reg, y_reg = linreg(df_max,x,y,z)

reg_3d(ax,xval,yval,mv_reg,x,y,z)

------3D Regression Results------ 

r2 =  0.3968
Equation:  My_d  =  -12.368  +  3.963 * RREV_threshold  +  0.24 * OF_y


In [22]:
# 2D REGRESSION PLOTs AND INFO
reg_2d(xval,yval,zval,x_reg,y_reg,x,y,z)

------2D Regression Results------ 

RREV_threshold results:
r2 =  0.3416
Equation:  My_d  =  -9.202  +  3.111 * RREV_threshold


OF_y results:
r2 =  0.0181
Equation:  My_d  =  4.661  +  -0.113 * OF_y


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


# DEFINE VARIABLES
x,y,z = 'RREV_threshold','OF_y','My_d'
X = df_max[x]
Y = df_max[y]
Z = df_max[z]
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Max Policy Relation')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
from scipy.odr import Model, Data, ODR
def f(B,x):
    return B[0]*x[0,:] + B[1]*x[1,:] + B[2]

length = len(xval)
xvalr,yvalr = xval.reshape(1,length),yval.reshape(1,length)
xy = np.concatenate((xvalr,yvalr),axis = 0)
linear = Model(f)
mydata = Data(xy,zval)
myodr = ODR(mydata,linear,beta0=[2,2,1])
myoutput = myodr.run()
#myoutput.pprint()

a = myoutput.beta[0]
b = myoutput.beta[1]
c = myoutput.beta[2]
zplot = a*xval + b*yval + c


ax.plot_trisurf(xval,yval,zplot)

r2 = 1- myoutput.res_var
print("My_d =  {c:.3f} + {a:.3f}*RREV + {b:.3f}*OF_y".format(a=a,b=b,c=c))
print("R2 = {r:.4f}".format(r=r2))

My_d =  -38.864 + 10.134*RREV + 0.797*OF_y
R2 = 0.9642
